In [1]:
%load_ext autoreload
%autoreload 2

In [196]:
from tfinterface.model_base import ModelBase
from tfinterface.reinforcement import ExperienceReplay
from tfinterface.utils import select_columns, soft_if, get_run, map_gradients
from phi.api import *
import tensorflow as tf
import random
from scipy.interpolate import interp1d
import numpy as np
import gym
from gym import wrappers
from tfinterface.reinforcement import ExpandedStateEnv
import os
import time


name = "dpg"

In [55]:
tf.slice?

In [101]:
def twodtensor2list(tensor,m,n):
    s = [[tf.slice(tensor,[j,i],[1,1]) for i in range(n)] for j in range(m)]
    fs = []
    for l in s:
        fs.extend(l)
    return fs

def grads_all_comp(y, shapey, x, shapex):
    yl = twodtensor2list(y,shapey[0],shapey[1])
    grads = [tf.gradients(yle,x)[0] for yle in yl]
    gradsp = tf.stack(grads)
    gradst = tf.reshape(gradsp,shape=(shapey[0],shapey[1],shapex[0],shapex[1]))
    return gradst


reduce?

In [134]:
a = tf.placeholder(tf.int32, [None], name='a')
sa = tf.one_hot(a, 4)
net = sa
net = tf.layers.dense(net, 8, activation=tf.nn.sigmoid, use_bias=False)
net = tf.layers.dense(net, 1, activation=tf.nn.sigmoid, use_bias=False)
ls = [ tf.gradients(net[i, 0], sa)[0] for i in range(4) ] |> sum
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

# sess.run(l)


sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run([- ls * sa ], feed_dict={
        a: [1, 2, 3, 0]
    }) 

[array([[ 0.        ,  0.00258938,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.01449001,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.01792335],
        [ 0.01353496,  0.        ,  0.        ,  0.        ]], dtype=float32)]

In [120]:
a = tf.placeholder(tf.int32, [None, 1], name='a')
b = tf.placeholder(tf.int32, [None, 4], name='a')

c = a * b

sess.run(c, {
        a: [[5], [3]], b: [[1, 2, 3, 4], [4, 3, 2, 1]]
    })

array([[ 5, 10, 15, 20],
       [12,  9,  6,  3]], dtype=int32)

In [126]:
1 + 2 |> (**)$(?, 2)

9

In [228]:
class Inputs(object):
    def __init__(self, n_states, scope):
        with tf.variable_scope(scope):
            self.episode_length = tf.placeholder(tf.int64, [], name='episode_length')

            self.s = tf.placeholder(tf.float32, [None, n_states], name='s')
            self.a = tf.placeholder(tf.int32, [None], name='a')
            self.r = tf.placeholder(tf.float32, [None], name='r')
            self.v1 = tf.placeholder(tf.float32, [None], name='Q1')
            self.done = tf.placeholder(tf.float32, [None], name='done')
            
            self.learning_rate = tf.placeholder(tf.float32, [], name='learning_rate')
            self.keep_prob = tf.placeholder(tf.float32, [], name='keep_prob')
            self.training = tf.placeholder(tf.bool, [], name='training')
            
            self.pi = tf.placeholder(tf.float32, [], name='pi')
            

class Critic(object):
    def __init__(self, actor, inputs, n_actions, n_states, y, batch_size, scope, actor_scope):
        with tf.variable_scope(scope):
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01)
            )

            sparse_a = tf.one_hot(inputs.a, n_actions)
            net = inputs.s
            net = tf.concat([net, sparse_a], 1)
            net = tf.layers.dense(net, 64, activation=tf.nn.relu, name="relu_layer", **ops)
#             net = tf.layers.dropout(net, inputs.keep_prob)
            net = tf.layers.dense(net, 64, activation=tf.nn.relu, name="relu_layer2", **ops)
            
            
            self.Q = tf.layers.dense(net, n_actions, name='Q', **ops)[:, 0]
            
#             with tf.name_scope("dQda"):
            self.dQda = [ tf.gradients(self.Q[i], sparse_a)[0] for i in range(batch_size) ] |> sum

            self.target = soft_if(inputs.done, inputs.r,  inputs.r + y * inputs.v1)

            self.error = self.target - self.Q
            self.loss = self.error |> tf.nn.l2_loss |> tf.reduce_mean

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)
            
            trainer = tf.train.AdamOptimizer(inputs.learning_rate)
            self.gradients = trainer.compute_gradients(self.loss, var_list=self.variables)
            self.gradients = map_gradients(tf.clip_by_norm$(?, 10), self.gradients)
            
            self.update = trainer.apply_gradients(self.gradients)

            avg_error, std_error = tf.nn.moments(self.error, [0])
            self.summaries = tf.summary.merge([
                tf.summary.scalar('loss', self.loss),
                tf.summary.scalar('avg_target', tf.reduce_mean(self.target)),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in self.variables ])),
                tf.summary.scalar('avg_error', avg_error),
                tf.summary.scalar('std_error', std_error),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    sparse_a,
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), self.variables[i]) for i in range(len(self.variables))
            ])
            
        with tf.variable_scope(actor_scope):
            
            actor.loss = -actor.P * self.dQda |> tf.reduce_mean
            
            trainer = tf.train.AdamOptimizer(inputs.learning_rate)
            actor.gradients = trainer.compute_gradients(actor.loss, var_list=actor.variables)
            actor.gradients = map_gradients(tf.clip_by_norm$(?, 10), actor.gradients)
            
            actor.update = trainer.apply_gradients(actor.gradients)
            
            actor.summaries = tf.summary.merge([
                tf.summary.scalar('loss', actor.loss),
                tf.summary.scalar('variables_sum', sum([ tf.reduce_sum(v) for v in actor.variables ])),
                tf.summary.histogram(
                    'avg_action', Pipe(
                    inputs.a,
                    Then(tf.one_hot, n_actions),
                    Then(tf.reduce_mean, axis=0)
                ))
            ]+[
                tf.summary.histogram('var{}'.format(i), actor.variables[i]) for i in range(len(actor.variables))
            ])
            
class Actor(object):
    def __init__(self, inputs, n_actions, n_states, y, scope):
        with tf.variable_scope(scope):
            ops = dict(
                trainable=True,
                kernel_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01),
                bias_initializer=tf.random_uniform_initializer(minval=0.0, maxval=0.01)
            )

            net = inputs.s

            net = tf.layers.dense(net, 64, activation=tf.nn.relu, name="relu_layer", use_bias=True, **ops)
            net = tf.nn.dropout(net, inputs.keep_prob)

            self.P = tf.layers.dense(net, n_actions, activation=tf.nn.softmax, name='P', use_bias=False, **ops)

            self.Pa = select_columns(self.P, inputs.a)

#             self.loss = - tf.log(tf.clip_by_value(self.Pa, 1e-3, 1.0)) * target_critic.error
#             self.loss = tf.reduce_mean(self.loss)

            self.variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=scope)

#             self.update = tf.train.AdamOptimizer(inputs.learning_rate).minimize(self.loss, var_list=self.variables)

            


In [232]:
class LunarLander(ModelBase):
    
    def define_model(self, n_actions, n_states, batch_size, y=0.98, buffer_length=50000, pi=0.1):
        self.global_max = float('-inf')

        self.replay_buffer = ExperienceReplay(max_length=buffer_length)


        with self.graph.as_default(), tf.device("cpu:0"):

            self.inputs = Inputs(n_states, "inputs")
    
            self.actor = Actor(self.inputs, n_actions, n_states, y, "actor")
            self.critic = Critic(self.actor, self.inputs, n_actions, n_states, y, batch_size, "critic", "actor")
            
            self.target_actor = Actor(self.inputs, n_actions, n_states, y, "target_actor")
            self.target_critic = Critic(self.target_actor, self.inputs, n_actions, n_states, y, batch_size, "target_critic", "target_actor")
            

            self.update = tf.group(self.critic.update, self.actor.update)

            self.episode_length_summary = tf.summary.scalar('episode_length', self.inputs.episode_length)

            self.summaries = tf.summary.merge([self.actor.summaries, self.critic.summaries, self.target_critic.summaries])

#             with tf.name_scope("update_target"):
            self.update_target = tf.group(*([
                t.assign_add(pi * (a - t)) for t, a in zip(self.target_critic.variables, self.critic.variables)
            ] + [
                t.assign_add(pi * (a - t)) for t, a in zip(self.target_actor.variables, self.actor.variables)
            ]))

    
    
    def predict_feed(self, S):
        return {
            self.inputs.s: S,
            self.inputs.keep_prob: 1.0,
            self.inputs.training: False
        }
    
    def predict(self, state, e = 0.0):
        predict_feed = self.predict_feed([state])
        actions = self.sess.run(self.actor.P, feed_dict=predict_feed)
        actions = actions[0]
        n = len(actions)

        if random.random() < e:
            return random.randint(0, n-1)
        else:
            return np.random.choice(n, p=actions)
    
    def fit_feed(self, S, A, R, Q1, Done, learning_rate, keep_prob):
        return {
            self.inputs.s: S,
            self.inputs.a: A,
            self.inputs.r: R,
            self.inputs.v1: Q1,
            self.inputs.done: Done,
            self.inputs.learning_rate: learning_rate,
            self.inputs.keep_prob: keep_prob,
            self.inputs.training: True
        }
    
    
    def fit(self, env, keep_prob=0.5, e=0.01, learning_rate=0.01, print_step=10, 
            update_target_step = 32, episodes=100000, max_episode_length=float('inf'), batch_size=32):
        
        r_total = 0.

        for episode in range(episodes):
            done = False
            ep_step = 0
            s = env.reset()
            episode_length = 0
            ep_reward = 0.
            
            while not done and ep_step <= max_episode_length:
                self.global_step += 1
                episode_length += 1
                ep_step += 1
                
                
                _learning_rate = learning_rate(self.global_step) if hasattr(learning_rate, '__call__') else learning_rate
                _e = e(self.global_step) if hasattr(e, '__call__') else e
                
                
                a = self.predict(s, e = _e)
                s1, r, done, info = env.step(a)
                r_total += r
                ep_reward += r
                
                
                self.replay_buffer.append((s, a, r, s1, float(done)))
                s = s1
                
                if len(self.replay_buffer) < batch_size:
                    continue
                
                
                S, A, R, S1, Done = self.replay_buffer.random_batch(batch_size).unzip()
                predict_feed = self.predict_feed(S1)
#                 Q1 = self.sess.run(self.target_critic.Q, feed_dict=predict_feed)
                Q1 = self.sess.run(self.target_critic.Q, feed_dict={
                    self.inputs.a: A,
                    self.inputs.s: S
                })

                
                fit_feed = self.fit_feed(S, A, R, Q1, Done, _learning_rate, keep_prob)
                _, summaries = self.sess.run([self.update, self.summaries], feed_dict=fit_feed)
                self.writer.add_summary(summaries, self.global_step)
                
                
                if self.global_step % update_target_step == 0:
                    self.sess.run(self.update_target)
                
                
                
                
            
            episode_length_summary = self.sess.run(self.episode_length_summary,
                                                   feed_dict={self.inputs.episode_length: episode_length})
            self.writer.add_summary(episode_length_summary, self.global_step)


            if ep_reward >= self.global_max:
                print("[MAX] Episode: {}, Length: {}, Reward: {}, buffer_len: {}".format(episode, episode_length, ep_reward, len(self.replay_buffer)))
                self.save(model_path = self.model_path + ".{score}".format(score = ep_reward))
                self.global_max = ep_reward


            if episode % print_step == 0 and episode > 0:
                avg_r = r_total / print_step
                actor_loss = self.sess.run(self.actor.loss, feed_dict=fit_feed)
                print("[NOR] Episode: {}, Length: {}, Avg Reward: {}, e: {}, Learning Rate: {}, buffer_len: {}".format(episode, episode_length, avg_r, _e, _learning_rate, len(self.replay_buffer)))
                print("Loss: {}".format(actor_loss))
                self.save()
                r_total = 0.


In [ ]:
run = get_run()
env = gym.make("LunarLander-v2")
env = wrappers.Monitor(env, "monitor/{run}".format(run = run))
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/models/{name}".format(path = os.getcwd(), name = name)
logs_path = "{path}/logs/{run}".format(path = os.getcwd(), run = run)
batch_size = 32

model = LunarLander(
    n_actions, n_states, batch_size, y=0.95, 
    buffer_length=500000,
    model_path = model_path,
    logs_path = logs_path,
    restore = False,
    pi = 0.002
)

model.critic.dQda, model.actor.P

[2017-03-18 14:52:52,577] Making new env: LunarLander-v2
[2017-03-18 14:52:52,580] DEPRECATION WARNING: env.spec.timestep_limit has been deprecated. Replace your call to `env.spec.timestep_limit` with `env.spec.tags.get('wrapper_config.TimeLimit.max_episode_steps')`. This change was made 12/28/2016 and is included in version 0.7.0
[2017-03-18 14:52:52,581] Creating monitor directory monitor/40


(<tf.Tensor 'critic/add_31:0' shape=(?, 4) dtype=float32>,
 <tf.Tensor 'actor/P/Softmax:0' shape=(?, 4) dtype=float32>)

In [ ]:
k = 40000.
model.fit(
    env, print_step=10, 
    episodes=int(1e5), max_episode_length=10000, batch_size=batch_size,
    learning_rate = 0.01, # lambda t: 0.05 * k / (k + t)
    e = interp1d([0, 300000], [0.4, 0.05], fill_value=0.05, bounds_error=False),
    keep_prob = 0.5,
    update_target_step = 1
)

[2017-03-18 14:52:55,282] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/dpg/monitor/40/openaigym.video.40.23423.video000000.mp4


[MAX] Episode: 0, Length: 71, Reward: -260.347299277, buffer_len: 71


[2017-03-18 14:52:56,639] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/dpg/monitor/40/openaigym.video.40.23423.video000001.mp4


[MAX] Episode: 2, Length: 64, Reward: -202.87439848, buffer_len: 234
[MAX] Episode: 3, Length: 83, Reward: -191.039266365, buffer_len: 317


[2017-03-18 14:53:00,085] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/dpg/monitor/40/openaigym.video.40.23423.video000008.mp4


...

[NOR] Episode: 30, Length: 127, Avg Reward: -197.55498628, e: 0.395529333333, Learning Rate: 0.01, buffer_len: 3833
Loss: 0.0
[NOR] Episode: 40, Length: 90, Avg Reward: -253.438403021, e: 0.394422166667, Learning Rate: 0.01, buffer_len: 4782
Loss: 0.0
[NOR] Episode: 50, Length: 90, Avg Reward: -218.0291636, e: 0.393411833333, Learning Rate: 0.01, buffer_len: 5648
Loss: 0.0
[NOR] Episode: 60, Length: 104, Avg Reward: -203.487916624, e: 0.392305833333, Learning Rate: 0.01, buffer_len: 6596
Loss: 0.0


[2017-03-18 14:53:22,862] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/dpg/monitor/40/openaigym.video.40.23423.video000064.mp4


[NOR] Episode: 70, Length: 100, Avg Reward: -288.091998268, e: 0.391262833333, Learning Rate: 0.01, buffer_len: 7490
Loss: 0.0
[NOR] Episode: 80, Length: 116, Avg Reward: -208.415860583, e: 0.390054166667, Learning Rate: 0.01, buffer_len: 8526
Loss: 0.0
[NOR] Episode: 90, Length: 109, Avg Reward: -327.674628503, e: 0.388910833333, Learning Rate: 0.01, buffer_len: 9506
Loss: 0.0
[NOR] Episode: 100, Length: 60, Avg Reward: -232.226460037, e: 0.3878585, Learning Rate: 0.01, buffer_len: 10408
Loss: 0.0
[NOR] Episode: 110, Length: 103, Avg Reward: -314.0914296, e: 0.386658, Learning Rate: 0.01, buffer_len: 11437
Loss: 0.0
[NOR] Episode: 120, Length: 86, Avg Reward: -234.591025184, e: 0.3855975, Learning Rate: 0.01, buffer_len: 12346
Loss: 0.0


[2017-03-18 14:53:42,828] Starting new video recorder writing to /home/cristian/data/neura-lab/q-learning/notebooks/lunar-lander/actor-critic/dpg/monitor/40/openaigym.video.40.23423.video000125.mp4


[NOR] Episode: 130, Length: 70, Avg Reward: -233.767467998, e: 0.384546333333, Learning Rate: 0.01, buffer_len: 13247
Loss: 0.0
[NOR] Episode: 140, Length: 76, Avg Reward: -249.280537292, e: 0.383519666667, Learning Rate: 0.01, buffer_len: 14127
Loss: 0.0
[NOR] Episode: 150, Length: 87, Avg Reward: -299.690477827, e: 0.382480166667, Learning Rate: 0.01, buffer_len: 15018
Loss: 0.0
[NOR] Episode: 160, Length: 76, Avg Reward: -331.495711536, e: 0.381465166667, Learning Rate: 0.01, buffer_len: 15888
Loss: 0.0
[NOR] Episode: 170, Length: 102, Avg Reward: -229.156550204, e: 0.380373166667, Learning Rate: 0.01, buffer_len: 16824
Loss: 0.0
[NOR] Episode: 180, Length: 99, Avg Reward: -172.548531054, e: 0.379270666667, Learning Rate: 0.01, buffer_len: 17769
Loss: 0.0
[NOR] Episode: 190, Length: 78, Avg Reward: -248.525776095, e: 0.378153, Learning Rate: 0.01, buffer_len: 18727
Loss: 0.0


In [ ]:

env = gym.make("LunarLander-v2")
env = ExpandedStateEnv(env, 3)
n_actions = env.action_space.n
n_states = env.observation_space.shape[0] * 3
model_path =  "{path}/{name}".format(path = os.getcwd(), name = name)
logs_path = "{path}/logs/".format(path = os.getcwd(), name = name)


model_run = LunarLander(
    n_actions, n_states,
    model_path = model_path,
    flush_secs = 3.0,
    restore = True
)

for i in range(100):
    s = env.reset()
    done = False
    total = 0.
    ep = 0
    while not done and ep < 700:
        ep += 1
        a = model_run.predict(s, 0.0)
        s, r, done, info = env.step(a)
        total += r
        env.render()
        time.sleep(0.01)
    
    print(total)
    
env.render(close=True)
